<h1 style="font-size: 40px; margin-bottom: 0px;">8.1 Exploring peak data and motif discovery</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Last week, we used <code>bowtie2</code> to align our raw reads to the hg19 reference genome. We then took those aligned reads and visualized the aligned reads using IGV. We then used <code>macs2</code> to identify significant and enriched peaks in our taz_1 and taz_2 datasets, and we were able to see how the reads pile up on both sides of the footprint of a DNA binding protein.

Today, first start off with some housekeeping as we're now generating a lot more data and also a lot more large data files. Then, we'll explore our peak data some more, making use of a program called <a href="http://homer.ucsd.edu/homer/" rel="noopener noreferrer" target="_blank"><u>HOMER (Hypergeometric Optimization of Motif EnRichment)</u></a>. First, we'll build HOMER into our Biology Hub server, then we will use it to take a look at the distribution of our reads around our summits. <strong>While you can run HOMER in your notebook, it'll be easier to run the installation and your HOMER commands in Terminal.</strong>

We'll also take a closer look at our peak data by looking at the intersection of our peaks, and taking a look at the concordance of the two TAZ replicates. Recall from lecture that Dr. Ingolia talked about analyzing the reproducibility of our data using the Irreproducibile Discovery Rate (IDR). We'll take a qualitative look at the IDR of our two TAZ replicates when we play around with the intersection of their called peaks.

Once we've identified our set of reproducible peaks, we can perform motif discovery on the overlapping peaks to see what are the binding sites recognized by the DNA binding protein(s) that TAZ is interacting with in order to regulate gene expression.

<strong>Learning objectives:</strong>

<ul>
    <li>General housekeeping</li>
    <li>Learn to install and use HOMER</li>
    <li>Learn to plot the peak profile</li>
    <li>Identify overlapping peaks</li>
    <li>Take a basic look at the concordance of the two replicates</li>
    <li>Pull data from peaks ranking highly in both replicates</li>
    <li>Identify binding motifs for TEAD+TAZ</li>
</ul>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<h1 style="font-size: 40px; margin-bottom: 0px;">Housekeeping and backing up files</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Since we're generating large data files, some of which are transitional files, we can do some general housekeeping before proceeding with today's lesson to help the Biology Hub maintainers keep storage needs and costs down. We'll play around in Terminal to poke around our server and see if we can reduce some of our storage footprint. We'll do this portion of the lab notebook in Terminal since it'll be easier to move around.

<h2>Checking directory sizes through Terminal</h2>

While we can hover over files in our File Browser to take a look at the file size, when we do the same thing with directories, we don't get any file size shown. So we'll make use of Terminal to take a look at how large our directories are overall and to identify potential areas where we can tidy up our server.

In [ ]:
%%bash

<h2>Back up <code>MCB201A_Su2025</code></h2>

For those of you who want to keep your MCB201A files, go ahead and download your <code>MCB201A_Su2025</code> directory, so that you have it saved as a compressed folder on your laptop.

<h2>Delete your server's copy of <code>MCB201A_Su2025</code></h2>

With <code>MCB201A_Su2025</code> backed up, let's go ahead and delete our server's copy of it.

In [ ]:
%%bash

<h2>Finding files known to be large</h2>

We know that we have particular output files that are known to be large, so we can use the <code>find</code> command to then identify those large files.

In [ ]:
%%bash

Then we can feed the output from the <code>find</code> command to the <code>du</code> command to get an output of the size of those files. We'll also modify how both commands are running in order to get a specific output from <code>find</code> that can be understood by <code>du</code> and to let <code>du</code> know that it will read in the file names from the standard input.

In [ ]:
%%bash

We can then also use the <code>find</code> command to then delete those files as well.

In [ ]:
%%bash

<h1 style="font-size: 40px; margin-bottom: 0px;">Installing HOMER</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

HOMER is a multifunctional program that can not only perform motif discovery, which is what we'll be primarily using it for, but it also has a lot of next-gen sequencing analysis functionality. For instance, it can also be used to call peaks, much like macs2.

HOMER was primarily developed by Chris Benner at UCSD.

I've placed a <code>configureHomer.pl</code> file in our shared class ChIP-seq directory. This file will be used to install the files needed for HOMER to run on your Biology Hub server.

First, you'll want to make a new directory in your home directory that will hold a <u>copy</u> of the <code>configureHomer.pl</code> file.

<h2>Change to home directory</h2>

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Make a new directory called <code>homer</code> in your home directory</h2>

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Enter <code>homer</code> directory and copy over <code>configureHomer.pl</code> to it</h2>

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Run <code>configureHomer.pl</code></h2>

Once you've copied over <code>configureHomer.pl</code>, you'll now need to run it in order to install the necessary files. Since this file uses Perl, another coding language, we will need to run it directly in command line.

In [ ]:
%%bash

There might be a few warnings, but there shouldn't be any errors.

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Update PATH to be able to run HOMER</h2>

We'll need to tell Terminal where to go to find the different files that HOMER uses. This temporarily adds the directories that HOMER uses to your <code>PATH</code> variable, but since the server reinitializes things, you'll need to go to your <code>homer</code> directory and re-run this line to update the <code>PATH</code> variable each time you start up your server.

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Install hg19 genome</h2>

Since our alignments were to the hg19 genome, we'll need to install this genome to use with HOMER and then prepare it for use later on when we perform motif discovery. First, let's look at what we currently have installed:

In [ ]:
%%bash

You should see that we don't have the hg19 genome installed, as indicated by the <code>-</code> by the name of the genome. So let's go ahead and install it:

In [ ]:
%%bash

Once you've installed the hg19 genome, you'll see a lot of new files under <code>~/homer/data/genomes/h19</code>, and if an analysis requires you to provide an input genome, you can simply specifiy <code>hg19</code>. Since it is a set of saved files and directories on your server, you don't have to reinstall it each time you start up your Biology Hub server.

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Preparse hg19 genome</h2>

Before we can use the hg19 genome though, we'll have to prepare it for use in motif discovery by preparsing the genome. This parses the genome into 50bp fragments that we can later use for motif discovery.

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h1 style="font-size: 40px; margin-bottom: 0px;">Plot peak profile</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

To get an idea of how all the reads are distributed around their summits, we can plot the peak profile for our two replicates. This allows us to visualize the average pileup around our peak summits to perform QC and to get an understanding of the binding footprint of our protein of interest.

<h2>Change directories to prepare to plot peak profiles</h2>

Change your current working directory to this week's directory and make a folder named <code>plot-profile</code>. Set your current working directory to the new folder you created.

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Use <code>annotatePeaks.pl</code> to generate histograms</h2>

HOMER has a program for peak annotation called <code>annotatePeaks.pl</code>, which like the other HOMER functions, you can run directly in Terminal. <a href="http://homer.ucsd.edu/homer/ngs/annotation.html" rel="noopener noreferrer" target="_blank"><u>Documentation for <code>annotatePeaks.pl</code> can be found here.</u></a>

<code>annotatePeaks.pl</code> is the workhorse of HOMER. It is multifunctional and can be used for a variety of purposes. It's primary purpose is, as it's name implies, to annotate the peaks identified through your peak calling program by identifying nearby genes, distance to the nearest transcription start sites (TSSs), and annotate your peaks with genomic features. It can also be used to generate histograms and heatmaps, which is one of the functions we'll be using it for here.

The syntax to use this program in command line is fairly straightforward; however, there are a number of different options that you can invoke that will change how the program runs, resulting in pretty different outputs.

If we take a look at the documentation to see how we can use this program:

<pre style="width: 450px; margin-top: 15px; margin-bottom: 15px; color: #000000; background-color: #EEEEEE; border: 1px solid; border-color: #AAAAAA; padding: 10px; border-radius: 15px; font-size: 12px;">annotatePeaks.pl &lt;peak file &vert; tss&gt; &lt;genome version&gt; &lbrack;options&rbrack;</pre>

We can either supply it with a peak file, such as our <code>.narrowPeak</code> or <code>*_summits.bed</code> output from <code>macs2</code>.

Alternatively, we can specify it to carry out a transcription start site (TSS)-centric analysis, in which case a peak file is not needed. Another way that we can alter how <code>annotatePeaks.pl</code> runs is by supplying it with additional options such as <code>-hist</code>, which will generate a histogram output if we supply it with coverage data in the form of a bedGraph file using the option <code>-bedGraph</code>.

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Obtain distribution of coverage/read counts</h2>

To obtain the distribution of coverage or reads around the summit for taz_1, you'll need your <code>10M_taz_1_summits.bed</code> file, as well as the bedgraph file that macs2 output, which contains the distribution of reads across the whole genome.

In [ ]:
%%bash

<h2>Let's breakdown the code:</h2>

<code>annotatePeaks.pl</code>

This calls up the annotatePeaks program within HOMER.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>~/MCB201B_F2025/Week_7/macs2/10M_taz_1_summits.bed</code>

Here, we are providing the annotatePeaks program with a bed file containing the coordinates for our peak summits because we're interested in taking a look at the coverage/distribution of read counts around our summits. 

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>hg19</code>

We specify the reference genome that we want the program to use. We can just provide it with the name of the reference genome <code>hg19</code>, and HOMER will know the directory containing the files for our reference genome that we installed earlier.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>-bedGraph ~/MCB201B_F2025/Week_7/macs2/10M_taz_1_treat_pileup.bdg</code>

While we've provided annotatePeaks with the genomic coordinates to each of our summits, we need to also provide it with information on the read coverage throughout the genome so that it can determine the distribution of reads around our summits. So here, we supply it with the bedGraph file that was generated by <code>macs2</code>, since this file contains information on read coverage for each location in the genome.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-size 2000</code>

The <code>-size</code> option specifies the size of our region of interest, centered around what we're interested in, which in our case are the peak summits. By specifying a size of 2000, we will be generating a histogram of a region 1kb upstream and downstream of the summit.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-hist 10</code>

The <code>-hist</code> option changes how annotatePeaks runs. This changes it essentially to "histogram" mode, where it will output (into Terminal) the distribution of read coverage around in our specified region of interest. The argument following the <code>-hist</code> option specifies the bin size for the histogram, so with <code>10</code>, we will be looking at the read distribution in intervals of 10.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>&gt; 10M_taz_1_summit_profile.txt</code>

We want annotatePeaks to save the output into a .txt file rather than dumping it into Terminal, so we redirect the output into <code>taz_1_summit_profile.txt</code>

<h1 style="font-size: 40px; margin-bottom: 0px;">Exercise set 1</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<h2>Exercise #1: Obtain coverage data around each summit for 10M_taz_2</h2>

Now see if you can run the <code>annotatePeaks.pl</code> program to generate the histogram for 10M_taz_2 using the same setup that we used for 10M_taz_1.

In [ ]:
%%bash

<h2>Exercise #2: Import HOMER outputs for both replicates</h2>

Now that we have both outputs, we'll make use of Python to take the output from HOMER to take a look at how the reads are distributed around the summits for both of our TAZ replicates.

First, you'll need to import both files into their own pandas DataFrames using the <code>pd.read_csv()</code> function.

<h2>Exercise #3: Simplify column headers</h2>

Take a look at the column headers. You'll see that they are a bit long with extra informaiton on our command line input that we don't need for plotting purposes. So replace your column header with the following:

<ol>
    <li><code>'distance_from_center'</code></li>
    <li><code>'coverage'</code></li>
    <li><code>'plus_tags'</code></li>
    <li><code>'minus_tags'</code></li>
</ol>

<h2>Exercise #4: Plot peak profiles</h2>

Now plot the peak profiles for both replicates as a line plot overlaid on the same plot with the X-axis as the distance from the center of the summit and the Y-axis as the coverage.

You can see in your plot that while the coverage pattern is similar between the two replicates, they vary slightly in their read depth, with taz_2 having overall higher coverage than taz_1. This is normally expected, since each replicate will have slightly different read counts. The important consideration is that the plot profiles look similar in their shape.

Looking at the shape of both of our peak profiles, we can see that it is a sharp, narrow peak profile which is characteristic of the peak profiles for transcription factors as they recognize and bind to specific sequences in the genome based on what their DNA binding domains recognize. By plotting the peak profile, we can now see that TEAD+TAZ recognize and bind to a specific sequence as opposed to just broadly recognizing DNA. We'll later on pull together our peaks to identify the motif(s) recognized by TEAD when its bound to TAZ. 

<h1 style="font-size: 40px; margin-bottom: 0px;">Generate heatmap of coverage/read density</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Among the documentation for <code>annotatePeaks.pl</code>, we can see that there is an additional option <code>-ghist</code> that we can specify, which will change the output generated by <code>annotatePeaks.pl</code>. In this case, rather than outputting the average coverage along all peaks, it will output a data matrix, where we are then able to see the coverage for each peak individually.

<h2>Create <code>heatmap</code> directory in <code>Week_8</code> and enter it</h2>

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Obtain data matrix for both TAZ replicates</h2>

In [ ]:
%%bash

So you can see that we run the code with the same input, except now we add in an additional option <code>-ghist</code>, which changes how it outputs the coverage data.

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Import our data matrices</h2>

Let's go ahead and import our data matrices, and then to simplify plotting, we can drop the <code>'Gene'</code> column since we don't need that for our heatmaps.

Let's do a quick sanity check.

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Generate heatmaps for both replicates</h2>

With everything imported, now we can make use of either <code>matplotlib</code> or <code>seaborn</code> to generate our heatmaps. For this example, we'll make use of <code>sns.heatmap()</code>. <a href="https://seaborn.pydata.org/generated/seaborn.heatmap.html" rel="noopener noreferrer">Documentation for <code>sns.heatmap()</code> can be found here.</u></a>

<h1 style="font-size: 40px; margin-bottom: 0px;">Plot TSS-centric read distribution</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

The multifunctionality of <code>annotatePeaks.pl</code> means that we can also perform a transcription start site (TSS)-centric analysis. In this case, we can supply the program with the <code>tss</code> argument along with our bedGraph files, so it can provide us with the distribution of reads around TSSs. This can help give us an idea into if our protein of interest is regulating its target genes via promoter binding or through more distant interactions.

<h2>Obtain read distributions around TSS</h2>

In [ ]:
%%bash

Here, we make minor alterations to how we ran the annotatePeaks program to instruct it to focus on TSSs and to provide a larger region of interest. So we provide the <code>tss</code> argument and increase the <code>-size</code> to <code>8000</code>. That way, we'll get an output containing the read distribution centered around TSSs, which we can plot like we did with the summit profile.

<h1 style="font-size: 40px; margin-bottom: 0px;">Exercise set 2</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<h2>Exercise #1: Obtain read distributions around TSS for 10M_taz_2</h2>

Now give it a try to update your input to instead analyze the distribution around TSSs for 10M_taz_2.

In [ ]:
%%bash

<h2>Exercise #2: Import your TSS analysis outputs</h2>

Like with our plot profiles earlier, update the column headers for convenience.

<ol>
    <li><code>'distance_from_center'</code></li>
    <li><code>'coverage'</code></li>
    <li><code>'plus_tags'</code></li>
    <li><code>'minus_tags'</code></li>
</ol>

<h2>Exercise #3: Plot the coverage around TSS for both replicates</h2>

Once you've imported your datasets, go ahead and plot the coverage around TSSs, using the same approach that you used for plotting the peak profile.

What does the distribution tell us about the way that TEAD+TAZ is potentially regulating gene transcription?

<h1 style="font-size: 40px; margin-bottom: 0px;">Determine distance from TSS</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

The results that we get when we look at the average coverage around TSSs seem to suggest that our peaks may be more distant than &pm;4kbp. What we can then do is take a look at the distribution of distances that our peaks are located from their nearest TSS. That can help give us a little bit more insight into our above result.

We'll once again make use of <code>annotatePeaks.pl</code> with slight modifications. If we take a look at <a href="http://homer.ucsd.edu/homer/ngs/annotation.html" rel="noopener noreferrer"><u>the documentation for <code>annotatePeaks.pl</code></u></a>, among one of the outputs for its basic annotate function is a column containing information on the distance to the nearest TSS, which is what we're interested in.

So we can use the basic annotate function of <code>annotatePeaks.pl</code> and focus on TSS annotations.

<h2>Create new directory <code>annotation</code> in this week's directory</h2>

Let's make a new directory <code>annotation</code>, and then enter it too.

In [ ]:
%%bash

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Annotate to find distance to nearest TSS for 10M_taz_1</h2>

Now lets modify how <code>annotatePeaks.pl</code> runs, and this time have it return to us the distances to TSSs for each peak summit.

In [ ]:
%%bash

<h2>Let's breakdown the code:</h2>

<code>annotatePeaks.pl</code>

This calls up the annotatePeaks program within HOMER.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>~/MCB201B_F2025/Week_7/macs2/10M_taz_1_summits.bed</code>

Here, we are providing the annotatePeaks program with a bed file containing the coordinates for our peak summits. This time we're interested in identifying the distances of our summits to the nearest TSS.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>hg19</code>

We indicate that we want to use the hg19 genome, which we have installed.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>-noann</code>

This option alters how the annotatePeaks program runs. By default the program will mark peak locations based on the genomic feature in which its located (intron, exon, TSS, CDS, etc), and the program will also identify the nearest gene and TSS. By providing the <code>-noann</code>, we are instructing the annotatePeaks program not to annotate peaks for genomic features, and instead, just focus on identifying nearby genes and TSSs. 

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>&gt; 10M_taz_1_annotation.txt</code>

We redirect the output data into a .txt file that we can then load into Python in order to pull relevant information.

<h1 style="font-size: 40px; margin-bottom: 0px;">Exercise Set 3</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<h2>Exercise #1: Obtain distance to nearest TSS for 10M_taz_2</h2>

Run <code>annotatePeaks.pl</code> now for 10M_taz_2 to identify the distance to the nearest TSS.

In [ ]:
%%bash

<h2>Exercise #2: Import your outputs</h2>

Now go ahead and import your output files containing the distances to the nearest TSS for each replicate.

Update the first column's header real quick to <code>'Peak ID'</code> for convenience.

<h2>Exercise #2: Calculate the absolute value of distances for each peak</h2>

Since our dataset contains both positive and negative distances for <code>'Distance to TSS'</code>, which just indicate relative position to the TSS either upstream or downstream, calculate the absolute value of the distances, since we're only really interested in how far away is TEAD+TAZ binding for this exercise.

Let's convert to kilobases for convenience.

<h2>Exercise #3: Plot a histogram of the distribution of distances</h2>

Now see if you can plot a histogram of the distribution of distances according to the following bins:

<ul>
    <li>From 0kb to 1kb</li>
    <li>From 1kb to 10kb</li>
    <li>From 10kb to 100kb</li>
    <li>>100kb</li>
</ul>

What do the distribution of peak distances to the nearest TSS suggest to us about how TAZ is regulating gene expression?

<h1 style="font-size: 40px; margin-bottom: 0px;">Find concordant peaks between two replicates</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Currently, we have two replicates with thousands of identified peaks that were found to be significant compared to the background levels (using our pooled IgG sequencing data). However, not all those peaks will be the same between replicates, due to biological and experimental variability, so we'll want to focus on just those peaks that are present in both our replicates for a more focused downstream analysis.

<h2>HOMER's mergePeaks program</h2>

To find peaks that are shared between our two replicates, we will make use of the <code>mergePeaks</code> command, and we'll provide it with the .narrowPeak files for both our replicates for analysis.

<a href="http://homer.ucsd.edu/homer/ngs/mergePeaks.html" rel="noopener noreferrer"><u>Documentation for the <code>mergePeaks</code> utility can be found here.</u></a>

If we take a look at the command line structure to use this program to pull overlapping peaks:

<pre style="width: 750px; margin-top: 15px; margin-bottom: 15px; color: #000000; background-color: #EEEEEE; border: 1px solid; border-color: #AAAAAA; padding: 10px; border-radius: 15px; font-size: 12px;">mergePeaks -d &lt;max distance to merge&gt; &lt;peak file 1&gt; &lt;peak file 2&gt; &lbrack;peak file N&rbrack; &lbrack;options&rbrack; -prefix &lt;base name&gt;</pre>

In [ ]:
%%bash

<h2>Let's breakdown the code:</h2>

<code>mergePeaks</code>

Here, we are calling up the mergePeaks program.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-d 100</code>

Here, we provide the program with the maximum distance that we would like it to merge peaks based on the distance between the peak centers. So peaks whose centers are greater than 100bp apart will not be considered overlapping.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>~/MCB201B_F2025/Week_7/macs2/10M_taz_1_peaks.narrowPeak</code>

The .narrowPeak file corresponding to our first replicate that we want it to pull peak information from.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>~/MCB201B_F2025/Week_7/macs2/10M_taz_1_peaks.narrowPeak</code>

The .narrowPeak file corresponding to our second replicate. mergePeaks will identify the overlapping peaks from the two files we supply it. We can also supply it with additional files, and it will analyze all of them.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-prefix merged</code>

Here, we supply it with the basename that we want it to add to all the files that it outputs. Based on how we set up the command, we will have three output files, two that correspond to unique peaks present in their respective replicates and one that contains the information on overlapping peaks, which is the file that we're most interested in.

<h2>Simplify output file names</h2>

If you take a look in your File Browser, you can see the three outputs, and they have rather long, unwieldy names, especially the file containing our overlapping peaks. So let's just directly in the File Browser rename the files for simplicity's sake.

Rename the file containing the overlapping peaks to: <code>merged_taz_peaks.narrowPeak</code>

<hr style="margin-left: 0px; border: 1px solid; border-color: #BBBBBB; width: 98%;"></hr>

<h2>Import <code>merged_taz_peaks.narrowPeak</code> as a DataFrame</h2>

Now let's take a look at the information contained within the file by importing into our Python notebook.

First, import the .narrowPeak file containing the merged peaks.

Now let's take a look to see if it imported okay.

So we can see that the column headers are a bit long, so we can shorten things for our sake later on by using the <code>pd.DataFrame.rename()</code> function.

And let's check to see if it updated properly:

Now that we have our overlapping peaks, it's not enough to just know which peaks are shared between our two replicates. We also need to know how reproducible these peaks are between our two replicates.

<h2>Assessing reproducibility</h2>

Dr. Ingolia mentioned in lecture that between replicates, there is variability in which peaks are called as well as their significance, which can be due to normal biological or experimental variability. <a href="https://doi.org/10.1101/gr.136184.111" rel="noopener noreferrer"><u>The standard way (set by the ENCODE and modeENCODE consortia)</u></a> of determining if the peaks that we identify through our peak calling program is to calculate the Irreproducible Discovery Rate (IDR), developed by <a href="https://doi.org/10.1214/11-AOAS466" rel="noopener noreferrer"><u>Li et al 2011</u></a>. 

The IDR framework allows for us to determine the reproducibility of our peaks (or in general the findings of high-throughput experiments) by comparing replicates to one another. IDR provides us with a quantitative assessment of when two replicates are no longer consistent with one another, which could point to findings that are less reproducible. In our case, that would suggest that those peaks may not necessarily be reproducible in repeated experiments. It's in a way conceptually similar to the FDR, where a lower IDR indicates a lower chance that the peak is irreproducible.

We'll just take a qualitative look at the concordance of our data, which will give us an idea of which peaks are probably more consistent between our replicates. And then we'll use those peaks for motif discovery.

To do this, we'll first need our original <code>.narrowPeak</code> files, so that we can rank the peaks according to their significance <code>'-log10(qvalue)'</code>. Then we can use our <code>merged_taz_peaks.narrowPeak</code> file containing our overlapping peaks to pull all the information only for the overlapping peaks. And then we can plot the two replicates against one another to see how correlated the ranks are for each of the overlapping peaks. We can also do the same for the significance of each overlapping peak.

<h1 style="font-size: 40px; margin-bottom: 0px;">Exercise Set 3</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<h2>Exercise #1: Import <code>.xls</code> files for both replicates</h2>

Let's import our original <code>.xls</code> files that was output by <code>macs2</code>, so we have all the information for each peak.

Let's check to see how the file imported.

<h2>Exercise #2: Rank data according to the <code>'-log10(qvalue)'</code></h2>

Now, we'll want to rank all the peaks according to their significance, with the greatest <code>'-log10(qvalue)'</code> having the assigned rank of 1. 

So we'll need to sort our data based on the <code>'-log10(qvalue)'</code>, and then we can assign ranks to each peak.

<h2>Guided Exercise: Pull together taz_1 peaks and taz_2 peaks based on the overlapping peaks</h2>

Now that we've ranked our individual replicates, we'll go ahead and then use our imported merged peaks to pull together the DataFrames for both replicates. To do this, we'll make use of the <code>pd.DataFrame.merge()</code> function, which will allow us to merge together two DataFrames based on a key or index. 

<a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html" rel="noopener noreferrer"><u>Documentation for <code>pd.DataFrame.merge()</code> can be found here.</u></a>

Digging into the documentation, we can see that the function is set up such that the DataFrame preceding the <code>.merge()</code> is the "left" DataFrame to be merged, while the DataFrame passed as an argument to the <code>.merge()</code> is the "right" DataFrame to be merged.

We can also provide it with the column containing the values that will act as the key that the merge will be based on with the <code>left_on</code> and <code>right_on</code> parameters. And if we want to distinguish the source of the columns after the merge, we can add suffixes to the column names (if the column name is shared between both DataFrames).

<h2>Exercise #3: Plot each replicates' ranks against each other</h2>

We can then plot the peaks' ranks against one another to visualize the reproducibility of our dataset. For this exercise, plot the ranks for taz_1 peaks on the x-axis and the ranks for the corresponding taz_2 peaks on the y-axis as a scatter plot.

What you should be able to see is a plot that begins to fan out as the value for the rank increases (so overall less significant peak), suggesting that those peaks identified in either set that do not have a high -log10(qvalue) may not be very reproducible. So in our more focused analysis, we will want to avoid those peaks and just look into those that consistently rank high (rank values closer to 1).

<h2>Exercise #4: Plot significance of two replicates against one another</h2>

We can also do the same for the <code>'-log10(qvalue)'</code> of our overlapping peaks.

To visually increase the spread of the lower significance samples, we can log transform the <code>'-log10(qvalue)'</code> using <code>np.log10()</code> when we plot each data point.

Alternatively, you can change the x- and y-axes to log scale.

Again, we can see that peaks that are highly significant in one replicate are likely to also be similarly significant in the second replicate, which suggest that they are more consistent between replicates. However, not all the peaks are necessarily highly consistent between our two replicates.

<h2>Exercise #5: Narrow down peaks to consistently high ranked peaks</h2>

For our motif discovery, we'll want to have a focused set of peaks to work with, so we can just focus on those peaks that are highly consistent between our two replicates. Normally, we would quantify the Irreproducible Discovery Rate (IDR), but here we'll just set a cut off of those peaks that are consistently in the top 500 in both replicates. See if you can pull just those highly reproducible peaks.

<h2>Exercise #6: Highlight the consistently high ranked peaks</h2>

Similar to what we did in notebook 7-2, let's highlight just the consistently high ranked peaks to visually distinguish them from the rest of the peaks in our scatter plot.

<h2>Exercise #7: Export your concordant peaks to a <code>.narrowPeak</code> file</h2>

Let's now export our top concordant peaks into a <code>.narrowPeak</code> file. Recall that the files that we've been working with are essentially plain text files with tabs as a delimiter to separate values that just have a custom file extension indicating that the file has a particular format/organization to its data. 

We can make use of <code>pd.DataFrame.read_csv()</code> to export our DataFrame containing our top concordant peaks. We don't need to export our <code>header</code> nor our <code>index</code>, so you'll have to modify how <code>.read_csv()</code> is running.

You can go ahead and export it to your <code>Week_8</code> directory.

<h1 style="font-size: 40px; margin-bottom: 0px;">Motif discovery</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Let's return to Terminal and HOMER. Here, we'll make use of the core function of HOMER, which is to identify motifs. To do this, we'll need to go back to Terminal to run the <code>findMotifsGenome.pl</code> program. 

<a href="http://homer.ucsd.edu/homer/ngs/peakMotifs.html" rel="noopener noreferrer"><u>Documentation for <code>findMotifsGenome.pl</code> can be found here.</u></a>

If we look into the documentation, we can see that the basic command line structure to use this code is:

<pre style="width: 600px; margin-top: 15px; margin-bottom: 15px; color: #000000; background-color: #EEEEEE; border: 1px solid; border-color: #AAAAAA; padding: 10px; border-radius: 15px; font-size: 12px;">findMotifsGenome.pl &lt;peak file&gt; &lt;genome version&gt; &lt;output directory&gt; -size &#35; &lbrack;options&rbrack;</pre>

So to run the <code>findMotifsGenome.pl</code> program, we will supply it with our <code>.narrowPeak</code> file containing our focused list of overlapping and reproducible peaks, the genome version, an output directory, a fragment size, and any additional options.

In [ ]:
%%bash

<h2>Let's break down the code:</h2>

<code>findMotifsGenome.pl</code>

We call up the findMotifsGenome program.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>~/MCB201B_F2025/Week_8/top_concordant_peaks.narrowPeak</code>

We provide it with our <code>.narrowPeak</code> file containing just our focused set of overlapping peaks.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>hg19</code>

We indicate the genome version that it will use to pull sequences and find motifs. Recall that when we first set up HOMER, we preparsed the genome for 50bp fragments, and now we will be using that preparsed genome for motif discovery.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>./</code>

We indicate the output directory as our current working directory.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-size 50</code>

We indicate the fragment size around the peak center where we want it to look for motifs. Here, it will look for motifs in the region &pm;50bp from the peak center.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-S 5</code>

This option specifies how many <i>de novo</i> motifs we want it to look for. The default option is 25, which is a lot. To keep the runtime down, we'll have it look for just 5 <i>de novo</i> motifs

<h2>Take a look at the identified motifs</h2>

Let's go ahead and take a look at the HTML output files to see what motifs we have identified within our peaks.

<h1 style="font-size: 40px; margin-bottom: 0px;">Visualize motif distributions</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Now let's take a look at all the distribution of all our various TEAD motifs around our peak center just to get an idea of where TEAD might be binding relative to the center of our peaks, which we know should theoretically map to where TEAD+TAZ binds to DNA.

To do this, we'll make use of annotatePeaks' histogram mode, but we'll provide it with our TEAD motifs that we've found through our analysis. What annotatePeaks will do is go through our focused set of peaks and determine the distribution of our TEAD motifs around the center of all the peaks.

<h2>Obtain motif distributions around peaks</h2>

To make things a little more convenient, we'll stay in our <code>motif</code> directory but specify our output into our <code>annotation</code> directory.

In [ ]:
%%bash

<h2>Let's break down the code:</h2>

<code>annotatePeaks.pl</code>

This calls up the annotatePeaks program.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>~/MCB201B_F2025/Week_8/top_concordant_peaks.narrowPeak</code>

This provides HOMER with our focused set of overlapping peaks.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>hg19</code>

We specify the genome version.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-m</code>

This tells the program that we want it to do an analysis of a specific motif or set of motifs. In our case, we are providing it with motifs that we want it to use to determine their distributions within our focused set of peaks.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>&lt;.motif files&gt;</code>

We provide the motif file(s) that we want the annotatePeaks program to use. We can supply it with a single motif file or more than one. In either case, the output will be a single file that contains information for each motif we provide it. Each motif will have 3 corresponding columns of data in the output: one for total counts, one for (+) counts, and one for (-) counts.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-size 2000</code>

We specify the size of the region of interest that we're interested in looking at motif distributions. With a size set to 2000, we will be looking at a region &pm;1000bp centered on our peaks.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>-hist 20</code>

We invoke the histogram mode of annotatePeaks and provide it with the bin size that it will use to determine the distribution of motifs in our specified region.

<hr style="border: 1px solid; border-color: #AAAAAA;"></hr>

<code>&gt; motif_distribution.txt</code>

We redirect the output to a file called <code>motif_distribution.txt</code> that we can then import into this Python notebook to plot.

<h1 style="font-size: 40px; margin-bottom: 0px;">Exercise Set 4</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

This exercise set will have you plot the motif distributions of your TEAD motifs that you analyzed with HOMER. This way, we can get an idea of how the motifs are distributed around our peaks, which can tell us more about how TEAD+TAZ is recognizing and binding to DNA.

<h2>Exercise #1: Import motif distribution file</h2>

<h2>Exercise #2: Plot motif distributions as a line plot</h2>

For this exercise, see if you can plot the motif distributions for TEAD1-4 and TEAD (family). While there are still unwieldy column headers, we only really care about the ones for total sites, so we can pull those out to plot. We can just update the first column header and pull the others as needed when plotting.

What does this motif distribution tell us about how TEAD+TAZ binds to DNA?

<h1 style="font-size: 40px; margin-bottom: 0px;">References</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<a href="https://www.nature.com/articles/ncb3216" rel="noopener noreferrer"><u>Zanconato et al 2015 Nat Cell Biol:</u></a> Genome-wide association between YAP/TAZ/TEAD and AP-1 at enhancers drives oncogenic growth

<a href="https://genome.cshlp.org/content/22/9/1813" rel="noopener noreferrer"><u>Landt et al 2012 Genome Res:</u></a> ChIP-seq guidelines and practices of the ENCODE and modENCODE consortia

<a href="https://projecteuclid.org/journals/annals-of-applied-statistics/volume-5/issue-3/Measuring-reproducibility-of-high-throughput-experiments/10.1214/11-AOAS466.full" rel="noopener noreferrer"><u>Li et al 2011 Ann Appl Stat:</u></a> Measuring reproducibility of high-throughput experiments